# Single cell analysis

In [ ]:
import pandas as pd
import scanpy as sc
import anndata as ad
import matplotlib as mpl

In [ ]:
sc.set_figure_params(color_map='Reds', figsize=[9.8, 11.8], dpi_save=300, format='pdf', fontsize=29)
sc.settings.figdir = '../03_figures'

In [ ]:
# the first 4 data sets are corrupted and were provided directly from the author of scPower
pbmcs_1 = sc.read_10x_mtx(
    'PBMC_pool_1_14_run1/',
    var_names='gene_symbols',
    cache=True)
pbmcs_2 = sc.read_10x_mtx(
    'PBMC_pool_1_14_run2/',
    var_names='gene_symbols',
    cache=True)
pbmcs_3 = sc.read_10x_mtx(
    'PBMC_pool_1_14_run3/',
    var_names='gene_symbols',
    cache=True)
pbmcs_4 = sc.read_10x_mtx(
    'PBMC_pool_1_14_overloaded/',
    var_names='gene_symbols',
    cache=True)
# the last 2 data sets are from GEO
pbmcs_5 = sc.read_10x_mtx(
    'GEO_all/GSE185714_RAW/',
    prefix='GSM5621963_pool_1_7_run1_',
    var_names='gene_symbols',
    cache=True)
pbmcs_6 = sc.read_10x_mtx(
    'GSE185714_RAW/',
    prefix='GSM5621964_pool_8_14_run1_',
    var_names='gene_symbols',
    cache=True)

In [ ]:
# add cell information
cell_ann_1 = pd.read_csv('PBMC_pool_1_14_run1/cellannotations.tsv', sep='\t', index_col=0)
pbmcs_1.obs = cell_ann_1
cell_ann_2 = pd.read_csv('PBMC_pool_1_14_run2/cellannotations.tsv', sep='\t', index_col=0)
pbmcs_2.obs = cell_ann_2
cell_ann_3 = pd.read_csv('PBMC_pool_1_14_run3/cellannotations.tsv', sep='\t', index_col=0)
pbmcs_3.obs = cell_ann_3
cell_ann_4 = pd.read_csv('PBMC_pool_1_14_overloaded/cellannotations.tsv', sep='\t', index_col=0)
pbmcs_4.obs = cell_ann_4
cell_ann_5 = pd.read_csv('GSE185714_RAW/GSM5621963_pool_1_7_run1_cellannotations.tsv', sep='\t', index_col=0)
pbmcs_5.obs = cell_ann_5
cell_ann_6 = pd.read_csv('GSE185714_RAW/GSM5621964_pool_8_14_run1_cellannotations.tsv', sep='\t', index_col=0)
pbmcs_6.obs = cell_ann_6

In [ ]:
# mitochondrial genes
pbmcs_1.var["mt"] = pbmcs_1.var_names.str.startswith("MT-")
pbmcs_2.var["mt"] = pbmcs_2.var_names.str.startswith("MT-")
pbmcs_3.var["mt"] = pbmcs_3.var_names.str.startswith("MT-")
pbmcs_4.var["mt"] = pbmcs_4.var_names.str.startswith("MT-")
pbmcs_5.var["mt"] = pbmcs_5.var_names.str.startswith("MT-")
pbmcs_6.var["mt"] = pbmcs_6.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(pbmcs_1, qc_vars=["mt"], percent_top=None, log1p=True, inplace=True)
sc.pp.calculate_qc_metrics(pbmcs_2, qc_vars=["mt"], percent_top=None, log1p=True, inplace=True)
sc.pp.calculate_qc_metrics(pbmcs_3, qc_vars=["mt"], percent_top=None, log1p=True, inplace=True)
sc.pp.calculate_qc_metrics(pbmcs_4, qc_vars=["mt"], percent_top=None, log1p=True, inplace=True)
sc.pp.calculate_qc_metrics(pbmcs_5, qc_vars=["mt"], percent_top=None, log1p=True, inplace=True)
sc.pp.calculate_qc_metrics(pbmcs_6, qc_vars=["mt"], percent_top=None, log1p=True, inplace=True)

The matrix.mtx files contain the raw data, however the cellannotations.csv files contain the results from the QC from the scPower paper ('demuxlet_doublet', 'demuxlet_ambigious', 'scrublet_doublet', 'mito_filter', 'count_filter'). If one of these is True, they are filtered out. Alternatively, the variable 'cell_type' is NA if the cell was filtered out. Therefore, I can use just this variable for filtering.

In [ ]:
pbmcs_1_qc = pbmcs_1[pd.notnull(pbmcs_1.obs['cell_type']), :]
pbmcs_2_qc = pbmcs_2[pd.notnull(pbmcs_2.obs['cell_type']), :]
pbmcs_3_qc = pbmcs_3[pd.notnull(pbmcs_3.obs['cell_type']), :]
pbmcs_4_qc = pbmcs_4[pd.notnull(pbmcs_4.obs['cell_type']), :]
pbmcs_5_qc = pbmcs_5[pd.notnull(pbmcs_5.obs['cell_type']), :]
pbmcs_6_qc = pbmcs_6[pd.notnull(pbmcs_6.obs['cell_type']), :]

In [ ]:
# concatenate all data sets
pbmcs_list = {
    "1_14_1": pbmcs_1_qc,
    "1_14_2": pbmcs_2_qc,
    "1_14_3": pbmcs_3_qc,
    "1_14_overloaded": pbmcs_4_qc,
    "1_7": pbmcs_5_qc,
    "8_14": pbmcs_6_qc
}
# the label argument adds the batch information
pbmcs_all_qc = ad.concat(pbmcs_list, label='batch', index_unique='_')

## Filter and normalize the data

In [ ]:
sc.pp.filter_genes(pbmcs_all_qc, min_cells=3)
sc.pp.normalize_total(pbmcs_all_qc, target_sum=1e4)
sc.pp.log1p(pbmcs_all_qc)

In [ ]:
# Regress out effects of total counts per cell and the percentage of mitochondrial genes expressed (taken from the scanpy 3k PBMC tutorial)
sc.pp.regress_out(pbmcs_all_qc, ['total_counts', 'pct_counts_mt'])

In [ ]:
# Scale each gene to unit variance. Clip values exceeding standard deviation 10.
sc.pp.scale(pbmcs_all_qc, max_value=10)

## Principal component analysis and data visualization

In [ ]:
sc.tl.pca(pbmcs_all_qc, svd_solver='arpack')

In [ ]:
# something went wrong when reading in the cellannotations.csv data
# (I suspect there were NAs so the columns weren't read in as boolean)
pbmcs_all_qc.obs['demuxlet_doublet'] = pbmcs_all_qc.obs.demuxlet_doublet == 'True'
pbmcs_all_qc.obs['demuxlet_ambigious'] = pbmcs_all_qc.obs.demuxlet_ambigious == 'True'
pbmcs_all_qc.obs['mito_filter'] = pbmcs_all_qc.obs.mito_filter == 'True'
pbmcs_all_qc.obs['count_filter'] = pbmcs_all_qc.obs.count_filter == 'True'

In [ ]:
sc.pp.neighbors(pbmcs_all_qc, n_neighbors=10, n_pcs=40)
sc.tl.umap(pbmcs_all_qc)

### Plots

In [ ]:
sc.pl.umap(pbmcs_all_qc, color=['cell_type'],
               save='_celltypes')

In [ ]:
sc.pl.umap(pbmcs_all_qc, color="SERPINF1",
           save='_serpinf1',
           color_map=mpl.colors.LinearSegmentedColormap.from_list("", ["lightgrey", "red"])
           )

In [ ]:
sc.pl.umap(pbmcs_all_qc, color="VEGFA",
           save='_vegfa',
           color_map=mpl.colors.LinearSegmentedColormap.from_list("", ["lightgrey", "red"]))